### Задание 1

In [1]:
import numpy as np
import pandas as pd

In [2]:
from sklearn.datasets import load_boston
boston = load_boston()

In [3]:
data = boston["data"]
feature_names = boston["feature_names"]
target = boston["target"]

X = pd.DataFrame(data, columns=feature_names)
y = pd.DataFrame(target, columns=["price"])

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
from sklearn.linear_model import LinearRegression

In [7]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [8]:
from sklearn.metrics import r2_score

In [9]:
r2_score(y_test, y_pred)

0.711226005748496

### Задание 2

In [10]:
from sklearn.ensemble import RandomForestRegressor

In [11]:
regr = RandomForestRegressor(n_estimators=1000, max_depth=12, random_state=42)
regr.fit(X_train, y_train.values[:, 0])
y_pred = regr.predict(X_test)
r2_score(y_test, y_pred)

0.87472606157312

In [12]:
# Для решения данной задачи модель, созданная с помощью метода RandomForestRegressor, работает лучше.

### Задание 3

In [13]:
RandomForestRegressor?

Init signature:
RandomForestRegressor(
    n_estimators=100,
    *,
    criterion='mse',
    max_depth=None,
    min_samples_split=2,
    min_samples_leaf=1,
    min_weight_fraction_leaf=0.0,
    max_features='auto',
    max_leaf_nodes=None,
    min_impurity_decrease=0.0,
    min_impurity_split=None,
    bootstrap=True,
    oob_score=False,
    n_jobs=None,
    random_state=None,
    verbose=0,
    warm_start=False,
    ccp_alpha=0.0,
    max_samples=None,
)
Docstring:     
A random forest regressor.

A random forest is a meta estimator that fits a number of classifying
decision trees on various sub-samples of the dataset and uses averaging
to improve the predictive accuracy and control over-fitting.
The sub-sample size is controlled with the `max_samples` parameter if
`bootstrap=True` (default), otherwise the whole dataset is used to build
each tree.

Read more in the :ref:`User Guide <forest>`.

Parameters
----------
n_estimators : int, default=100
    The number of trees in the fore

In [14]:
pd.Series(regr.feature_importances_, index=feature_names).sort_values(ascending=False)

LSTAT      0.415847
RM         0.402682
DIS        0.063973
CRIM       0.031676
PTRATIO    0.018081
AGE        0.014299
NOX        0.014269
B          0.012451
TAX        0.011525
INDUS      0.007138
RAD        0.005281
ZN         0.001543
CHAS       0.001236
dtype: float64

In [15]:
# Наибольшую важность показывают признаки LSTAT и RM.